In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

c:\Users\sachi\OneDrive\Desktop\AppliedML_Nyc_TAXI_prediction\AppliedML_NYC_taxidata_pred\AppliedML_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 12:12:01,239 INFO: Initializing external client
2025-03-06 12:12:01,240 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 12:12:02,322 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Fetching data from 2025-02-05 17:12:01.227927+00:00 to 2025-03-06 16:12:01.227927+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.11s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 18:00:00+00:00,2,0
1,2025-02-05 19:00:00+00:00,2,0
2,2025-02-05 20:00:00+00:00,2,0
3,2025-02-05 21:00:00+00:00,2,0
4,2025-02-05 22:00:00+00:00,2,0
...,...,...,...
175135,2025-03-06 12:00:00+00:00,263,148
175136,2025-03-06 13:00:00+00:00,263,125
175137,2025-03-06 14:00:00+00:00,263,112
175138,2025-03-06 15:00:00+00:00,263,121


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[us]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 12:12:13,122 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 12:12:13,122 INFO: Initializing external client
2025-03-06 12:12:13,122 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 12:12:13,827 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Fetching data from 2025-02-05 17:12:13.122716+00:00 to 2025-03-06 16:12:13.122716+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.08s) 


In [11]:
current_date

Timestamp('2025-03-06 17:12:13.122716+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,2,0,1,0,210,2025-02-10 03:00:00
1,3,0,0,0,1,2,1,0,0,2,...,4,1,0,0,0,2,0,0,216,2025-03-03 10:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,147,2025-02-23 16:00:00
3,3,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,217,2025-03-06 13:00:00
4,0,50,1,0,6,34,32,9,3,2,...,2,41,37,4,35,3,0,22,261,2025-02-19 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,27,13,16,19,19,47,23,41,43,13,...,37,26,21,15,11,40,0,27,125,2025-02-27 08:00:00
248,90,22,2,51,32,17,22,6,54,76,...,4,21,22,29,24,34,2,131,158,2025-02-06 16:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,47,2025-03-04 20:00:00
250,16,126,95,86,29,34,86,65,156,40,...,37,116,172,154,15,3,17,134,107,2025-02-20 14:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 12:12:30,705 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 12:12:30,705 INFO: Initializing external client
2025-03-06 12:12:30,705 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 12:12:31,372 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [15]:
predictions

,pickup_location_id,predicted_demand
0,210,0.0
1,216,1.0
2,147,0.0
3,217,1.0
4,261,22.0
...,...,...
247,125,23.0
248,158,77.0
249,47,0.0
250,107,107.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
124,161,280.0
216,237,263.0
130,236,260.0
174,132,227.0
73,239,225.0
101,68,160.0
141,170,159.0
230,79,153.0
31,140,146.0
217,230,135.0
